In [1]:
!git clone https://github.com/Seung-hwanSong/LG2024_DTKNN.git #코랩 사용

'git' is not recognized as an internal or external command,
operable program or batch file.


# [머신러닝 기반 시계열 분석 1 실습]
# Decision Tree & k-NN 
## [Decision Tree: Regression Tree - Example]

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- 함수 ( ) 안에서 shift+tab: arguments description. shift+tab+tab은 길게 볼 수 있도록

## 1. 모듈 불러오기

import 불러올 패키지명 as 그 패키지를 파이썬에서 사용할 이름

In [2]:
""" 데이터 전처리 """
import pandas as pd
import numpy as np


""" 모델 생성, 학습, 평가 """
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

""" 시각화 """
import matplotlib.pyplot as plt

## 2. 데이터 불러오기

- Stock Market Prediction using Decision Tree

In [3]:
# apple 주식 data 불러오기
df = pd.read_csv('/content/LG2024_DTKNN/data/AAPL_data.csv')
# df = pd.read_csv('../data/AAPL_data.csv') #로컬

FileNotFoundError: [Errno 2] No such file or directory: '/content/LG2024_DTKNN/data/AAPL_data.csv'

In [ ]:
# data 상위 5개 행 살펴보기
df.head()

In [ ]:
# data 정보 확인
df.info()

In [ ]:
# data 요약 통계량 확인
df.describe()

In [ ]:
# data 크기 확인
df.shape

## 2.1 Exploratory Data Analysis(EDA)

In [ ]:
# Apple stock data의 opening price 시각화
plt.figure(figsize = (16,8))
plt.title('Apple stock')
plt.xlabel('Days')
plt.ylabel('Opening Price USD ($)')
plt.plot(df['open'])
plt.show()

In [ ]:
# Apple stock data의 low price 시각화
plt.figure(figsize = (16,8))
plt.title('Apple stock')
plt.xlabel('Days')
plt.ylabel('Low Price USD ($)')
plt.plot(df['low'])
plt.show()

In [ ]:
# Apple stock data의 close price 시각화
plt.figure(figsize=(16,8))
plt.title('Apple')
plt.xlabel('Days')
plt.ylabel('Closing Price USD ($)')
plt.plot(df['close'])
plt.show()

In [ ]:
# close price data를 새로운 dataframe df2에 저장
df2 = df['close']
df2 = pd.DataFrame(df2)

## 2.2 Prediction 100 days into the future

In [ ]:
# 현재 data에서 100시점 만큼 미래를 예측하는 것이 목적이므로 데이터를 100시점 만큼 앞으로 당김 -> df2['prediction']에 저장
# df2['Prediction'] 끝에서부터 100시점은 Not A Number(NAN) 처리됨
future_days = 100
df2['Prediction'] = df2['close'].shift(-future_days)
df2

In [ ]:
# df2['Prediction']는 df2['close']를 100시점만큼 이전으로 평행이동한 값임을 plot을 통해 확인
plt.figure(figsize=(16,8))
plt.title('Apple Stock')
plt.xlabel('Days')
plt.ylabel('Price USD ($)')
plt.plot(df2['close'])
plt.plot(df2['Prediction'])
plt.legend(['close','Prediction'])
plt.show()

In [ ]:
# df['close']의 끝시점으로부터 100시점 이전까지를 제외한 후 X array에 저장
X = np.array(df2.drop(['Prediction'], 1))[:-future_days]
print(X)

In [ ]:
# df['Prediction']의 끝시점으로부터 100시점 이전까지를 제외한 후 y array에 저장
y = np.array(df2['Prediction'])[:-future_days]
print(y)

In [ ]:
# 즉, y는 X 각 시점의 100시점 이후의 값을 의미하며 이를 모델의 X, y값으로 대입
plt.figure(figsize=(16,8))
plt.title('Apple Stock')
plt.xlabel('Days')
plt.ylabel('Closing Price USD ($)')
plt.plot(X)
plt.plot(y)

## 3. Linear and Decision Tree Regression 

In [ ]:
# X,y를 train data와 test data로 split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
# Linear tree regression과 Decision Tree Regression에 train data 대입
tree = DecisionTreeRegressor().fit(x_train, y_train)
lr = LinearRegression().fit(x_train, y_train)

In [ ]:
# test를 위해 x_future에 X의 끝시점에서 100이전까지의 값을 x_future에 저장
x_future = pd.DataFrame(X).tail(future_days)
x_future

In [ ]:
# x_future의 Decision Tree Regression 결과
tree_prediction = tree.predict(x_future)
print(tree_prediction)

In [ ]:
# x_future의 Linear Regression 결과
lr_prediction = lr.predict(x_future)
print(lr_prediction)

In [ ]:
# 비교를 위해 valid_linear dataframe에 실제 값(['close'])과 예측값('Prediction')을 대입
# valid_tree['predictions']는 decision tree 예측값을 의미함
valid_lr = df2[X.shape[0]:]
valid_lr['Predictions'] = lr_prediction
valid_lr

In [ ]:
# linear regression 예측 결과와 실제 결과 비교를 위해 plot 그림
plt.figure(figsize=(16,8))
plt.title("Model")
plt.xlabel('Days')
plt.ylabel('Close Price USD ($)')
plt.plot(df2['close'])
plt.plot(valid_lr[['close', 'Predictions']])
plt.legend(["Original", "Valid", 'Predicted'])
plt.show()

In [ ]:
# 비교를 위해 valid_tree dataframe에 실제 값(['close'])과 예측값('Prediction')을 대입
# valid_tree['predictions']는 decision tree 예측값을 의미함
valid_tree = df2[X.shape[0]:]
valid_tree['Predictions'] = tree_prediction
valid_tree

In [ ]:
# Decision Tree Regression 결과와 실제 결과 비교를 위해 plot 그림
plt.figure(figsize=(16,8))
plt.title("Model")
plt.xlabel('Days')
plt.ylabel('Close Price USD ($)')
plt.plot(df2['close'])
plt.plot(valid_tree[['close', 'Predictions']])
plt.legend(["Original", "Valid", 'Predicted'])
plt.show()

In [ ]:
# Linear Regression과 실제 값의 정확한 결과 비교를 위해 끝에서 100시점 부분 확대
plt.figure(figsize=(16,8))
plt.title("Model Comparison")
plt.xlabel('Days')
plt.ylabel('Close Price USD ($)')
plt.plot(valid_lr[['close']], color = 'b')
plt.plot(valid_lr[['Predictions']], color = 'darkorange', linestyle = '--')
plt.plot(valid_tree[['Predictions']], color = 'r', linestyle = '--')
plt.legend(["Original", "Linear Regression", 'Decision Tree'])

In [ ]:
RMSE_lr = mean_squared_error(valid_lr[['close']].values, valid_lr[['Predictions']].values)**0.5
RMSE_lr

In [ ]:
RMSE_tree = mean_squared_error(valid_tree[['close']].values, valid_tree[['Predictions']].values)**0.5
RMSE_tree

# EOD